In [2]:
library(igraph)

In [3]:
version

               _                           
platform       x86_64-apple-darwin15.6.0   
arch           x86_64                      
os             darwin15.6.0                
system         x86_64, darwin15.6.0        
status                                     
major          3                           
minor          4.4                         
year           2018                        
month          03                          
day            15                          
svn rev        74408                       
language       R                           
version.string R version 3.4.4 (2018-03-15)
nickname       Someone to Lean On          

In [4]:
load("Q12data.RData")
# g <- the graph of movies in part 2
# communities <- the communities result in part 2
# actorId2pagerank <- actor's pagerank, e.g. pagerank of actor 0 = actorId2pagerank[1]
# movieId2actorIds <- movies' actor list, e.g. actor list of movie 0 = movieId2actorIds[[1]]

In [5]:
movie_id_targets <- c(10321, 39182, 78995)
movie_name_targets <- c("Batman v Superman: Dawn of Justice (2016)", "Mission: Impossible - Rogue Nation (2015)", "Minions (2015)")

### Q13
actor weight: the average of the top 5 ratings of the actor's movies

features: top 5 actor weights for each movie

model: average the features

In [ ]:
actor_ratings <- list()
for (i in 1:vcount(g)){
    if (is.na(V(g)$rating[i])) { next }
    
    movie_id <- as.numeric(V(g)$name[i])
    movie_rating <- V(g)$rating[i]    
    actors <- movieId2actorIds[[movie_id+1]]
    for (actorId in actors) {
        if (length(actor_ratings) <= actorId) {
            actor_ratings[[actorId+1]] <- c(movie_rating)
        }
        else {
            actor_ratings[[actorId+1]] <- c(actor_ratings[[actorId+1]], movie_rating)
        }
    }
}

actor_weight <- c()
for (ratings in actor_ratings) {
    if (is.null(ratings)) {
        weight <- 0
    }
    else {
        weight <- mean(sort(ratings, decreasing=TRUE)[1:5], na.rm=TRUE)
    }
    actor_weight <- c(actor_weight, weight)
}

In [31]:
saveRDS(actor_weight, "actor_weight.rds")

In [27]:
predict_rating <- function(movieId) {
    actors <- movieId2actorIds[[movieId+1]]
    weights <- c()
    for (actorId in actors) {
        weights <- c(weights, actor_weight[actorId+1])
    }
    return(mean(sort(weights, decreasing=TRUE)[1:5], na.rm=TRUE))
}

In [28]:
truth <- c()
prediction <- c()
for (i in 1:vcount(g)){
    if (is.na(V(g)$rating[i])) { next }
    
    movie_id <- as.numeric(V(g)$name[i])
    movie_rating <- V(g)$rating[i]
    truth <- c(truth, movie_rating)
    prediction <- c(prediction, predict_rating(movie_id))
}

In [29]:
cat("RMSE:", sqrt(mean((truth - prediction)^2)))

RMSE: 2.10236

In [30]:
for (i in 1:3) {
    movie_id <- movie_id_targets[i]
    cat(movie_name_targets[i], '\n')
    node_id <- which(V(g)$name == movie_id)
    rating_truth <- V(g)$rating[node_id]
    cat("Ground truth rating:", rating_truth, '\n')
    cat("Predicted rating:", predict_rating(movie_id), '\n', '\n')
}

Batman v Superman: Dawn of Justice (2016) 
Ground truth rating: NA 
Predicted rating: 8.511 
 
Mission: Impossible - Rogue Nation (2015) 
Ground truth rating: NA 
Predicted rating: 8.26 
 
Minions (2015) 
Ground truth rating: NA 
Predicted rating: 9.212 
 
